In [9]:
import pickle as pkl
import rasterio as rio
import numpy as np
import os

In [10]:
# modelname = "1684923533_model_57_012345678"
# modelname = "1685098495_model_54_01234567"
modelname = "final_snowprob_1692788122_model_53_01269"

pred_path = f"/home/abetatos/Escritorio/TFM_Dataset/5.Preds/{modelname}/"
hs_path = "/home/abetatos/Escritorio/TFM_Dataset/4.DatasetPost/HS/"

In [11]:

for filename in os.listdir(pred_path): 
    print(filename)
    with open(f"{pred_path}/{filename}/HS_pred05.pickle", "rb") as f: 
        mapping = pkl.load(f)
        shape = mapping.shape

    with rio.open(f"{hs_path}/{filename}/HS.tif", "r") as f: 
        profile = f.profile
        arr = f.read().squeeze()
        
    new_mapping = np.ones_like(arr)*-9999.
    
    new_mapping[0:shape[0], 0:shape[1]] = mapping

    with rio.open(f"{pred_path}/{filename}/0.5HS_pred.tif", "w", **profile) as f:
        f.write(new_mapping, 1)

    with rio.open(f"{pred_path}/{filename}/0.5HS_diff.tif", "w", **profile) as f:
        diff_array = np.where((arr != -9999.) & (new_mapping != -9999.), abs(arr-new_mapping), -9999.)
        f.write(diff_array, 1)

    with rio.open(f"{pred_path}/{filename}/0.5HS_sigma.tif", "w", **profile) as f:
        sigma_array = np.where(diff_array != -9999., np.divide(diff_array, arr + 1e-10), -9999.)
        f.write(sigma_array, 1)
    
    print(sigma_array[sigma_array!=-9999.].mean(), diff_array[diff_array!=-9999.].mean())
    print(sigma_array[sigma_array!=-9999.].max(), diff_array[diff_array!=-9999.].max())
    # break

HS_Davos_20170316_2m_epsg2056
5905838.5 0.60325634
29773408000.0 8.455338
